In [1]:
import os

os.chdir("../")

In [2]:
from dataclasses import dataclass
from pathlib import Path


@dataclass(frozen=True)
class ModelEvaluationConfig:
    root_dir: Path
    data_path: Path
    model_path: Path
    tokenizer_path: Path
    metric_file_name: Path

In [3]:
from src.constants import *
from src.utils import read_yaml, create_directories

In [4]:
class ConfigurationManager:
    def __init__(
        self, config_filepath=CONFIG_FILE_PATH, params_filepath=PARAMS_FILE_PATH
    ):

        self.config = read_yaml(config_filepath)
        self.params = read_yaml(params_filepath)

        create_directories([self.config.artifacts_root])

    def get_model_evaluation_config(self):
        config = self.config.model_evaluation

        create_directories([config.root_dir])

        model_evaluation_config = ModelEvaluationConfig(
            root_dir=config.root_dir,
            data_path=config.data_path,
            model_path=config.model_path,
            tokenizer_path=config.tokenizer_path,
            metric_file_name=config.metric_file_name,
        )

        return model_evaluation_config

In [5]:
from transformers import TrainingArguments, Trainer

from transformers import BartForConditionalGeneration, BertTokenizer
from datasets import load_dataset, load_from_disk, load_metric

import torch
import pandas as pd
from tqdm import tqdm

[2024-06-09 20:03:54,180: WARNING: module_wrapper: From c:\Users\IT STORE\anaconda3\Lib\site-packages\keras\src\losses.py:2976: The name tf.losses.sparse_softmax_cross_entropy is deprecated. Please use tf.compat.v1.losses.sparse_softmax_cross_entropy instead.
]
[2024-06-09 20:03:54,391: INFO: utils: Note: NumExpr detected 12 cores but "NUMEXPR_MAX_THREADS" not set, so enforcing safe limit of 8.]
[2024-06-09 20:03:54,391: INFO: utils: NumExpr defaulting to 8 threads.]


In [10]:
class ModelEvaluation:
    def __init__(self, config):
        self.config = config

    def generate_batch_sized_chunks(self, list_of_elements, batch_size):
    """
    Yield successive batch-sized chunks from the list of elements.

    Parameters:
    - list_of_elements (List): The list of elements to be divided into chunks.
    - batch_size (int): The size of each chunk.

    Yields:
    - List: A batch-sized chunk of elements from the input list.

    Example:
    >>> list_of_numbers = [1, 2, 3, 4, 5, 6, 7, 8, 9, 10]
    >>> batch_generator = generate_batch_sized_chunks(list_of_numbers, 3)
    >>> next(batch_generator)
    [1, 2, 3]
    >>> next(batch_generator)
    [4, 5, 6]
    >>> next(batch_generator)
    [7, 8, 9]
    >>> next(batch_generator)
    [10]
    """
    for i in range(0, len(list_of_elements), batch_size):
        yield list_of_elements[i : i + batch_size]

    def calculate_metric_on_test_dataset(
        dataset,
        metric,
        model,
        tokenizer,
        batch_size=16,
        device=("cuda" if torch.cuda.is_available() else "cpu"),
        column_text="article",
        column_summary="highlights",
    ):
    """
    This function calculates the metric on a test dataset using a given model and tokenizer.

    Parameters:
    - dataset: The dataset on which to calculate the metric.
    - metric: The metric object to use for calculation.
    - model: The model to generate summaries.
    - tokenizer: The tokenizer to use for encoding the input data.
    - batch_size (optional): The batch size for processing the dataset. Default is 16.
    - device (optional): The device to use for model and tensor computations. Default is 'cuda' if available, else 'cpu'.
    - column_text (optional): The column name in the dataset containing the text data. Default is 'article'.
    - column_summary (optional): The column name in the dataset containing the summary data. Default is 'highlights'.

    Returns:
    - score: The computed metric score.
    """

    # Split the dataset into batches for processing
    article_batches = list(
        self.generate_batch_sized_chunks(dataset[column_text], batch_size)
    )
    target_batches = list(
        self.generate_batch_sized_chunks(dataset[column_summary], batch_size)
    )

    # Process each batch and calculate the metric
    for article_batch, target_batch in tqdm(
        zip(article_batches, target_batches), total=len(article_batches)
    ):

        # Encode the input data
        inputs = tokenizer(
            article_batch,
            max_length=1024,
            truncation=True,
            padding="max_length",
            return_tensors="pt",
        )

        # Generate summaries using the model
        summaries = model.generate(
            input_ids=inputs["input_ids"].to(device),
            attention_mask=inputs["attention_mask"].to(device),
            length_penalty=0.8,
            num_beams=8,
            max_length=128,
        )

        # Decode the generated summaries
        decoded_summaries = [
            tokenizer.decode(
                s, skip_special_tokens=True, clean_up_tokenization_spaces=True
            )
            for s in summaries
        ]

        # Remove any empty spaces from the decoded summaries
        decoded_summaries = [d.replace("", " ") for d in decoded_summaries]

        # Add the predictions and references to the metric
        metric.add_batch(predictions=decoded_summaries, references=target_batch)

    # Compute the final metric score
    score = metric.compute()
    return score

    def evaluate(self):
        device = "cuda" if torch.cuda.is_available() else "cpu"
        tokenizer = BertTokenizer.from_pretrained(self.config.tokenizer_path)
        model = BartForConditionalGeneration.from_pretrained(self.config.model_path).to(
            device
        )

        dataset_samsum_pt = load_from_disk(self.config.data_path)

        rouge_names = ["rouge1", "rouge2", "rougeL", "rougeLsum"]

        rouge_metric = load_metric("rouge")

        score = self.calculate_metric_on_test_ds(
            dataset_samsum_pt["test"][0:10],
            rouge_metric,
            model,
            tokenizer,
            batch_size=2,
            column_text="dialogue",
            column_summary="summary",
        )

        rouge_dict = dict((rn, score[rn].mid.fmeasure) for rn in rouge_names)

        df = pd.DataFrame(rouge_dict, index=["bert-large-cnn"])
        df.to_csv(self.config.metric_file_name, index=False)

In [ ]:
try:
    config = ConfigurationManager()
    model_evaluation_config = config.get_model_evaluation_config()
    model_evaluation_config = ModelEvaluation(config=model_evaluation_config)
    model_evaluation_config.evaluate()

except Exception as e:
    raise e